In [1]:
import sqlite3

import sys
import os
PROJECT_ROOT = os.path.abspath('../..')
sys.path.append(PROJECT_ROOT)

# AJUSTA ESTOS IMPORTS a tu proyecto real:
from machine_learning.data_collectors import build_ml_dataframe, build_supervised_dataset
from machine_learning.evaluators import eval_regression_extended

from machine_learning.tcn.train_tcn import train_eval_tcn, TrainTCNConfig
from machine_learning.artifacts import load_model_artifact_auto

from machine_learning.data_collectors import (
    build_ml_dataframe,
    build_supervised_dataset,
    time_split_masks,
    purged_ts_cv_splits,
    TARGET_HORIZONS,
    TARGET_LOOKBACKS,
    parse_feat_lag
)
from database_tier1 import TARGET_STOCKS
from python_scripts.LLM_analysis.preprocess_store_database import get_connection



[DB_PATH_DEFAULT] /home/aprohack/Desktop/all_folders/Investings_project/app/data/stock_data.db
{'foreign_keys': 1, 'journal_mode': 'wal', 'synchronous': 1}


In [2]:
# 1) Cargar datos (long: symbol, timestamp, OHLCV, indicadores, etc.)
conn = get_connection()

timeframe = "1Day"
symbols = TARGET_STOCKS

start = None
end = None

include_indicators = False
indicator_names = []
# indicator_names = ['RSI_14', 'BBB_20_2.0', 'BBP_20_2.0', 'ATRr_14']

include_economic_indicators = False
econ_indicator_names = []
# econ_indicator_names = ['CPI', 'UNEMPLOYMENT']

include_fmp = False
fmp_feature_names = []
keep_fmp_asof_date = False
fmp_prefix = 'fmp'

# -----------------------
# ELIGE LOOKBACK AQUÍ
# -----------------------
lookback = TARGET_LOOKBACKS[2]  # <-- cámbialo

# 3 horizontes baseline (puedes editar)
#horizons = [5, 20, 60]
horizon = TARGET_HORIZONS[2]

base_feature_cols = ['open', 'high', 'low', 'close', 'volume', 'trade_count']

lags_by_feature = None
default_lags = lookback


feature_cols = base_feature_cols + indicator_names + econ_indicator_names + fmp_feature_names


print(f"lb={lookback}, h={horizon}")





lb=120, h=20


In [3]:
df = build_ml_dataframe(
    conn,
    symbols=symbols,
    timeframe="1Day",
    start="2015-01-01",
    end="2025-12-31",
    include_indicators=True,
    include_econ=True,
    include_fmp=False,
)

# 2) Definir feature_cols (excluir columnas no-feature)
non_feature_cols = {"symbol", "timestamp", "timeframe"}
feature_cols = [c for c in df.columns if c not in non_feature_cols]

In [ ]:
# (Opcional) sanity-check single-horizon dataset (verifica meta['target_timestamp'])
X_wide_5, y_5, meta_5 = build_supervised_dataset(
    df,
    feature_cols=feature_cols,
    lookback=60,
    horizon=5,
    price_col="close",
    group_col="symbol",
    timestamp_col="timestamp",
)
print("Single-horizon check:", X_wide_5.shape, y_5.shape, meta_5.columns)

# 3) Entrenar + evaluar + guardar artifact (multi-horizon en el mismo modelo)
cfg = TrainTCNConfig(
    lookback=60,
    horizons=(5, 20, 60),
    batch_size=256,
    dropout=0.10,
    channels=(64, 64, 64, 64),
    kernel_size=3,
    lr=1e-3,
    weight_decay=1e-4,
    max_epochs=50,
    patience=10,
    grad_clip=1.0,
    periods_per_year=252,   # diario
    run_base_dir="runs",
    run_name=None,
)

out = train_eval_tcn(
    df,
    feature_cols=feature_cols,
    build_supervised_dataset_fn=build_supervised_dataset,
    eval_fn=eval_regression_extended,
    cfg=cfg,
)

print("Artifact guardado en:", out["run_dir"])
print("Horizons:", out["horizons"])
print("Keys metrics:", out["metrics"].keys())



Single-horizon check: (198102, 420) (198102,) Index(['symbol', 'timestamp', 'target_timestamp'], dtype='object')


/home/aprohack/Desktop/all_folders/Investings_project/app/investenv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


[epoch 001] train_loss=0.008085 | val_rmse_mean=0.090712 (h5:0.040973, h20:0.079231, h60:0.151932) | best=0.090712 @epoch 1 | no_improve=0
[epoch 002] train_loss=0.006621 | val_rmse_mean=0.092036 (h5:0.042118, h20:0.080064, h60:0.153926) | best=0.090712 @epoch 1 | no_improve=1
[epoch 003] train_loss=0.006577 | val_rmse_mean=0.090218 (h5:0.040991, h20:0.079035, h60:0.150630) | best=0.090218 @epoch 3 | no_improve=0
[epoch 004] train_loss=0.006526 | val_rmse_mean=0.089591 (h5:0.039675, h20:0.077803, h60:0.151295) | best=0.089591 @epoch 4 | no_improve=0
[epoch 005] train_loss=0.006480 | val_rmse_mean=0.091655 (h5:0.041064, h20:0.077576, h60:0.156324) | best=0.089591 @epoch 4 | no_improve=1
[epoch 006] train_loss=0.006448 | val_rmse_mean=0.091345 (h5:0.039652, h20:0.081338, h60:0.153045) | best=0.089591 @epoch 4 | no_improve=2
[epoch 007] train_loss=0.006407 | val_rmse_mean=0.092037 (h5:0.042899, h20:0.078584, h60:0.154628) | best=0.089591 @epoch 4 | no_improve=3
[epoch 008] train_loss=0.00

/home/aprohack/Desktop/all_folders/Investings_project/app/investenv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [5]:
# 4) Cargar artifact (auto)
artifact = load_model_artifact_auto(out["run_dir"], map_location="cpu")
model = artifact["model"]
scaler = artifact["scaler"]
config = artifact["config"]
print("Loaded model_type:", config.get("model_type"))

Loaded model_type: tcn_regressor
